In [1]:
# Import modules
import pandas as pd
import numpy as np
from numpy import log as ln
from numpy import log10
from numpy_financial import pmt
import os
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
from pandas.tseries.offsets import MonthEnd
from functools import reduce

# Set up Pandas defaults
pd.options.display.float_format = '{:.4f}'.format
pd.set_option("display.max_columns", None)

# Define helper function to create directory
def create_folder(the_path):
    "Create directory if nonexistent."
    if not os.path.isdir(the_path):
        os.mkdir(the_path)

create_folder('outputs')

census_end_year = int(input("What year was the most recent ACS 5-Year census data pulled from? "))
acs_1_year = int(input("What year was the most recent ACS 1-Year census data pulled from? "))

In [2]:
# Read in BiggerScore V1
biggerscore_v1 = pd.read_csv("outputs/ranked_msas.csv", dtype={'msa_code':str})

# Remove any places with missing price data
biggerscore_v1 = biggerscore_v1[biggerscore_v1['ACS_1_Year_Median_Price'].notna()]

# Remove places without HouseCanary Data
biggerscore_v1 = biggerscore_v1[biggerscore_v1['housecanary_rentpriceratio'].notna()]


biggerscore_v1.head(3)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,hc_median_rent_upper_bound,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply,Median_Prop_Tax_Rate,Normalized Total Employment,5-Year Job Growth x Normalized,RANK_5-Year Household Growth,RANK_5-Year Population Growth,RANK_5-Year Job Growth,RANK_5-Year Job Growth x Normalized,RANK_Income,RANK_5-Year Income Growth,RANK_Unemployment_Rate,RANK_Permits_as_Percent_of_Total_Units,RANK_1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_ACS_1_Year_Rent-Price_Ratio,RANK_housecanary_rentpriceratio,RANK_1-Year Price Forecast,RANK_ACS_1_Year_Median_Price,RANK_insurance,RANK_Median_Prop_Tax_Rate,RANK_Vacancy_Rate
0,"Barnstable Town, MA",12700,0.0522,0.0184,229436.0000,0.0726,0.0066,0.3944,634700.0000,0.5556,1398.0000,0.2572,0.2203,165068.0000,"Barnstable Town, MA",2,70900,111000.0000,-0.0009,54992.6000,0.2270,4.1000,2023.0000,418.0000,12.0000,25.0000,104.0000,547.0000,76.4168,2.1938,4.5704,19.0128,0.0033,723600.0000,2709.0000,3667.0000,0.0037,0.0487,0.0041,MA,Metropolitan Statistical Area,3658.0000,1998.0000,0.0151,0.0058,0.0094,-0.0008,198,306,87,88,217,211,197,307,350,9,13,248,17,326,252,1
1,"Atlantic City, NJ",12100,0.0669,0.0181,369795.0000,0.0063,0.0012,0.3519,350000.0000,0.5960,1132.0000,0.2358,0.3234,231311.0000,"Atlantic City-Hammonton, NJ",3,12100,133200.0000,-0.0075,50598.6000,0.1706,6.8000,2023.0000,299.0000,44.0000,44.0000,40.0000,383.0000,78.0679,11.4883,11.4883,10.4439,0.0017,361100.0000,2121.0000,2867.0000,0.0059,0.0806,0.0107,NJ,Metropolitan Statistical Area,6373.0000,1894.0000,0.0165,0.0182,0.0116,-0.0086,238,111,70,52,138,147,13,353,357,120,151,357,110,329,21,2
2,"Naples, FL",34940,0.1349,0.0199,380221.0000,0.0657,0.0199,0.3178,595500.0000,0.6108,1867.0000,0.6420,0.3135,229814.0000,"Naples-Marco Island, FL",3,34940,173000.0000,0.1556,51055.1600,0.1194,3.7000,2023.0000,2923.0000,30.0000,106.0000,589.0000,3618.0000,80.7905,0.8292,2.9298,16.2797,0.0157,663600.0000,3066.0000,4154.0000,0.0046,0.0259,-0.1459,FL,Metropolitan Statistical Area,2807.0000,4984.0000,0.0041,0.0047,0.0155,0.2413,346,294,358,313,146,92,266,90,217,100,51,79,23,18,309,3


### Read in Tax Rankings

In [3]:
tax_rankings = pd.read_csv("../datasets_manual_download/State_Taxes/state_corporate_tax_ranking.csv")
tax_rankings['2014-2024 Rank Change'] = tax_rankings['2014 Rank'] - tax_rankings['2024 Rank']
tax_rankings.sort_values("2014-2024 Rank Change", ascending=False).head(5)

### Read in state abbrs and combine
state_abbr = pd.read_csv("../helper_datasets/states_abbr.csv")
tax_rankings = tax_rankings.merge(state_abbr, how='left', on='State')

# Rename
tax_rankings = tax_rankings[['Abbreviation','2024 Rank','2014-2024 Rank Change']]
tax_rankings.rename(columns={
    'Abbreviation':'state',
    '2024 Rank':'Corporate_Tax_Ranking',
    '2014-2024 Rank Change':'Corporate_Tax_Ranking_10_Year_Change'
    }, inplace=True)

tax_rankings.sort_values("Corporate_Tax_Ranking_10_Year_Change", ascending=False).head(5)

,state,Corporate_Tax_Ranking,Corporate_Tax_Ranking_10_Year_Change
32,NC,5,21
30,NM,13,20
14,IA,29,19
13,IN,12,16
15,KS,21,14


In [4]:
tax_rankings.sort_values("Corporate_Tax_Ranking", ascending=True).head(5)

,state,Corporate_Tax_Ranking,Corporate_Tax_Ranking_10_Year_Change
49,WY,1,0
40,SD,1,0
24,MO,3,1
35,OK,4,7
32,NC,5,21


### Add in Investor Friendliness

In [5]:
landlord_friendly = pd.read_csv("../unique_datasets/landlord_friendliness_by_msa.csv",
                                dtype={'msa_code':str})

landlord_friendly.drop(columns=['msa_name'], inplace=True)

landlord_friendly.head(3)

,msa_code,landlord_friendly_or_not
0,10180,Landlord Friendly
1,10420,Landlord Friendly
2,10500,Landlord Friendly


### Combine to make BiggerScore V2

In [6]:
biggerscore_v2 = biggerscore_v1.merge(tax_rankings, how='left', on='state')
biggerscore_v2 = biggerscore_v2.merge(landlord_friendly, how='left', on='msa_code')

biggerscore_v2.head(5)

,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,hc_median_rent_upper_bound,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply,Median_Prop_Tax_Rate,Normalized Total Employment,5-Year Job Growth x Normalized,RANK_5-Year Household Growth,RANK_5-Year Population Growth,RANK_5-Year Job Growth,RANK_5-Year Job Growth x Normalized,RANK_Income,RANK_5-Year Income Growth,RANK_Unemployment_Rate,RANK_Permits_as_Percent_of_Total_Units,RANK_1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_ACS_1_Year_Rent-Price_Ratio,RANK_housecanary_rentpriceratio,RANK_1-Year Price Forecast,RANK_ACS_1_Year_Median_Price,RANK_insurance,RANK_Median_Prop_Tax_Rate,RANK_Vacancy_Rate,Corporate_Tax_Ranking,Corporate_Tax_Ranking_10_Year_Change,landlord_friendly_or_not
0,"Barnstable Town, MA",12700,0.0522,0.0184,229436.0000,0.0726,0.0066,0.3944,634700.0000,0.5556,1398.0000,0.2572,0.2203,165068.0000,"Barnstable Town, MA",2,70900,111000.0000,-0.0009,54992.6000,0.2270,4.1000,2023.0000,418.0000,12.0000,25.0000,104.0000,547.0000,76.4168,2.1938,4.5704,19.0128,0.0033,723600.0000,2709.0000,3667.0000,0.0037,0.0487,0.0041,MA,Metropolitan Statistical Area,3658.0000,1998.0000,0.0151,0.0058,0.0094,-0.0008,198,306,87,88,217,211,197,307,350,9,13,248,17,326,252,1,36,-4,Landlord Unfriendly
1,"Atlantic City, NJ",12100,0.0669,0.0181,369795.0000,0.0063,0.0012,0.3519,350000.0000,0.5960,1132.0000,0.2358,0.3234,231311.0000,"Atlantic City-Hammonton, NJ",3,12100,133200.0000,-0.0075,50598.6000,0.1706,6.8000,2023.0000,299.0000,44.0000,44.0000,40.0000,383.0000,78.0679,11.4883,11.4883,10.4439,0.0017,361100.0000,2121.0000,2867.0000,0.0059,0.0806,0.0107,NJ,Metropolitan Statistical Area,6373.0000,1894.0000,0.0165,0.0182,0.0116,-0.0086,238,111,70,52,138,147,13,353,357,120,151,357,110,329,21,2,48,-11,Landlord Unfriendly
2,"Naples, FL",34940,0.1349,0.0199,380221.0000,0.0657,0.0199,0.3178,595500.0000,0.6108,1867.0000,0.6420,0.3135,229814.0000,"Naples-Marco Island, FL",3,34940,173000.0000,0.1556,51055.1600,0.1194,3.7000,2023.0000,2923.0000,30.0000,106.0000,589.0000,3618.0000,80.7905,0.8292,2.9298,16.2797,0.0157,663600.0000,3066.0000,4154.0000,0.0046,0.0259,-0.1459,FL,Metropolitan Statistical Area,2807.0000,4984.0000,0.0041,0.0047,0.0155,0.2413,346,294,358,313,146,92,266,90,217,100,51,79,23,18,309,3,11,2,Landlord Friendly
3,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,197200.0000,0.1129,43691.4400,0.3566,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,346000.0000,1884.0000,2412.0000,0.0054,0.0218,-0.0039,SC,Metropolitan Statistical Area,725.0000,3219.0000,-0.0060,0.0023,0.0179,0.2017,349,364,330,304,59,332,39,2,29,177,113,52,128,177,366,4,6,6,Landlord Friendly
4,"Daphne, AL",19300,0.1927,0.0414,233420.0000,0.1478,0.0277,0.2742,307000.0000,0.5841,1089.0000,0.3105,0.3547,125113.0000,"Daphne-Fairhope-Foley, AL",2,19300,89200.0000,0.1150,42389.3600,0.2067,3.1000,2023.0000,3316.0000,16.0000,32.0000,1015.0000,4363.0000,76.0028,0.3667,0.7334,23.2638,0.0349,369200.0000,2037.0000,2655.0000,0.0055,0.0321,-0.0105,AL,Metropolitan Statistical Area,718.0000,3798.0000,0.0066,0.0023,0.0073,0.0839,

### Create a Relative Growth Index

In [7]:
# Create min-max normalization
biggerscore_v2['Normalized Total Employment'] = (
    (biggerscore_v2['Jobs'] - biggerscore_v2['Jobs'].min())
    /
    (biggerscore_v2['Jobs'].max() - biggerscore_v2['Jobs'].min())
    )

# Create growth x normalized column
biggerscore_v2['5-Year Job Growth x Normalized'] = 100 * biggerscore_v2['5-Year Job Growth'] * biggerscore_v2['Normalized Total Employment']

biggerscore_v2


,msa_name,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,msa_name_original,Population_Size_Category,msa_code_bls,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,hc_median_rent_upper_bound,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply,Median_Prop_Tax_Rate,Normalized Total Employment,5-Year Job Growth x Normalized,RANK_5-Year Household Growth,RANK_5-Year Population Growth,RANK_5-Year Job Growth,RANK_5-Year Job Growth x Normalized,RANK_Income,RANK_5-Year Income Growth,RANK_Unemployment_Rate,RANK_Permits_as_Percent_of_Total_Units,RANK_1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_ACS_1_Year_Rent-Price_Ratio,RANK_housecanary_rentpriceratio,RANK_1-Year Price Forecast,RANK_ACS_1_Year_Median_Price,RANK_insurance,RANK_Median_Prop_Tax_Rate,RANK_Vacancy_Rate,Corporate_Tax_Ranking,Corporate_Tax_Ranking_10_Year_Change,landlord_friendly_or_not
0,"Barnstable Town, MA",12700,0.0522,0.0184,229436.0000,0.0726,0.0066,0.3944,634700.0000,0.5556,1398.0000,0.2572,0.2203,165068.0000,"Barnstable Town, MA",2,70900,111000.0000,-0.0009,54992.6000,0.2270,4.1000,2023.0000,418.0000,12.0000,25.0000,104.0000,547.0000,76.4168,2.1938,4.5704,19.0128,0.0033,723600.0000,2709.0000,3667.0000,0.0037,0.0487,0.0041,MA,Metropolitan Statistical Area,3658.0000,1998.0000,0.0151,0.0058,0.0087,-0.0008,198,306,87,88,217,211,197,307,350,9,13,248,17,326,252,1,36,-4,Landlord Unfriendly
1,"Atlantic City, NJ",12100,0.0669,0.0181,369795.0000,0.0063,0.0012,0.3519,350000.0000,0.5960,1132.0000,0.2358,0.3234,231311.0000,"Atlantic City-Hammonton, NJ",3,12100,133200.0000,-0.0075,50598.6000,0.1706,6.8000,2023.0000,299.0000,44.0000,44.0000,40.0000,383.0000,78.0679,11.4883,11.4883,10.4439,0.0017,361100.0000,2121.0000,2867.0000,0.0059,0.0806,0.0107,NJ,Metropolitan Statistical Area,6373.0000,1894.0000,0.0165,0.0182,0.0109,-0.0081,238,111,70,52,138,147,13,353,357,120,151,357,110,329,21,2,48,-11,Landlord Unfriendly
2,"Naples, FL",34940,0.1349,0.0199,380221.0000,0.0657,0.0199,0.3178,595500.0000,0.6108,1867.0000,0.6420,0.3135,229814.0000,"Naples-Marco Island, FL",3,34940,173000.0000,0.1556,51055.1600,0.1194,3.7000,2023.0000,2923.0000,30.0000,106.0000,589.0000,3618.0000,80.7905,0.8292,2.9298,16.2797,0.0157,663600.0000,3066.0000,4154.0000,0.0046,0.0259,-0.1459,FL,Metropolitan Statistical Area,2807.0000,4984.0000,0.0041,0.0047,0.0148,0.2297,346,294,358,313,146,92,266,90,217,100,51,79,23,18,309,3,11,2,Landlord Friendly
3,"Myrtle Beach, SC",34820,0.1375,0.0578,356578.0000,0.1496,0.0340,0.3114,316800.0000,0.6271,1108.0000,0.3867,0.3497,206764.0000,"Myrtle Beach-Conway-North Myrtle Beach, SC",3,34820,197200.0000,0.1129,43691.4400,0.3566,5.5000,2023.0000,11228.0000,78.0000,159.0000,1789.0000,13176.0000,85.2155,0.5920,1.2067,13.5777,0.0637,346000.0000,1884.0000,2412.0000,0.0054,0.0218,-0.0039,SC,Metropolitan Statistical Area,725.0000,3219.0000,-0.0060,0.0023,0.0171,0.1933,349,364,330,304,59,332,39,2,29,177,113,52,128,177,366,4,6,6,Landlord Friendly
4,"Daphne, AL",19300,0.1927,0.0414,233420.0000,0.1478,0.0277,0.2742,307000.0000,0.5841,1089.0000,0.3105,0.3547,125113.0000,"Daphne-Fairhope-Foley, AL",2,19300,89200.0000,0.1150,42389.3600,0.2067,3.1000,2023.0000,3316.0000,16.0000,32.0000,1015.0000,4363.0000,76.0028,0.3667,0.7334,23.2638,0.0349,369200.0000,2037.0000,2655.0000,0.0055,0.0321,-0.0105,AL,Metropolitan Statistical Area,718.0000,3798.0000,0.0066,0.0023,0.0065,0.0753,

# Now Turn into the 4 Buckets with appropriate weights:

- Cashflow Score (50%)
- Investor Friendliness (10%)
- Demand Score (20%)
- Supply Score (20%)

In [15]:
display_msa = biggerscore_v2.copy()

# Rank Weights
rank_weights = {

    'Demand':{
        'RANK_5-Year Household Growth': 2,
        'RANK_5-Year Population Growth': 2,
        'RANK_5-Year Job Growth': 2,
        'RANK_5-Year Job Growth x Normalized': 5,
        'RANK_Income': 2,
        'RANK_5-Year Income Growth': 1,
        'RANK_Unemployment_Rate': 2,
        'RANK_Vacancy_Rate': 2,
        'Population_Size_Category': 250,
        'RANK_1-Year Price Forecast': 2,
    },

    'Supply': {
        'RANK_Permits_as_Percent_of_Total_Units': 2,
        'RANK_1-Year_HH_Growth_Minus_Percent_New_Supply': 2,
    },

    'Cashflow': {
        'RANK_housecanary_rentpriceratio': 3,
        'RANK_ACS_1_Year_Median_Price': 2,
        'RANK_insurance': 5,
        'RANK_Median_Prop_Tax_Rate': 5,
    }
    
}



### Make cashflow index
cashflow_score = 0
display_msa['Cashflow_Rank_Sum'] = 0
for col in rank_weights['Cashflow']:
    cashflow_score += display_msa[col].max() * rank_weights['Cashflow'][col]
    display_msa['Cashflow_Rank_Sum'] += display_msa[col] * rank_weights['Cashflow'][col]
# Create Cashflow Score
display_msa['Cashflow_Score'] = display_msa['Cashflow_Rank_Sum'] / cashflow_score

### Make a supply index
supply_score = 0
display_msa['Supply_Rank_Sum'] = 0
for col in rank_weights['Supply']:
    supply_score += display_msa[col].max() * rank_weights['Supply'][col]
    display_msa['Supply_Rank_Sum'] += display_msa[col] * rank_weights['Supply'][col]
# Create Cashflow Score
display_msa['Supply_Score'] = display_msa['Supply_Rank_Sum'] / supply_score

### Make a demand index
demand_score = 0
display_msa['Demand_Rank_Sum'] = 0
for col in rank_weights['Demand']:
    demand_score += display_msa[col].max() * rank_weights['Demand'][col]
    display_msa['Demand_Rank_Sum'] += display_msa[col] * rank_weights['Demand'][col]
# Create Cashflow Score
display_msa['Demand_Score'] = display_msa['Demand_Rank_Sum'] / demand_score


# Create a Landlord Friendliness Modifier
display_msa['Landlord_Friendliness_Modifier'] = np.where(
    display_msa['landlord_friendly_or_not'] == 'Landlord Friendly',
    0.1,
    np.where(
        display_msa['landlord_friendly_or_not'] == 'Landlord Semi-Friendly',
        0,
        -0.1
    )
)

### Now create a BiggerScore_v2 based on new weights
ranking_weights_2 = {
    'Cashflow_Score': 4,
    'Supply_Score': 2,
    'Demand_Score': 2,
}




### Create the basic sums of the ranks
max_score = 0
display_msa['Total_Rank_Sum'] = 0
for main_col in rank_weights:
    for col in rank_weights[main_col]:
        max_score += display_msa[col].max() * rank_weights[main_col][col]
        display_msa['Total_Rank_Sum'] += display_msa[col] * rank_weights[main_col][col]
# Create BiggerScore
display_msa['BiggerScore'] = (display_msa['Total_Rank_Sum'] / max_score) 
display_msa['BiggerScore'] += (display_msa['Landlord_Friendliness_Modifier'] * display_msa['BiggerScore'])



### Now create ordinal categories -- perhaps group MSAs into Tiers
def create_ordinal_metrics(
        df,
        name_of_ordinal_column,
        column_to_measure,
        ascending=False
):
    

    if ascending:
        df[name_of_ordinal_column] = np.where(
            df[column_to_measure]>=df[column_to_measure].quantile(0.75),
            1,
            np.where(
                df[column_to_measure]>=df[column_to_measure].quantile(0.5),
                2,
                np.where(
                    df[column_to_measure]>=df[column_to_measure].quantile(0.25),
                    3,
                    4
                )
            )
        )
    else:
        df[name_of_ordinal_column] = np.where(
        df[column_to_measure]<=df[column_to_measure].quantile(0.25),
        1,
        np.where(
            df[column_to_measure]<=df[column_to_measure].quantile(0.5),
            2,
            np.where(
                df[column_to_measure]<=df[column_to_measure].quantile(0.75),
                3,
                4
            )
        )
    )

    return df

display_msa = create_ordinal_metrics(display_msa, "Job_Growth_Tier", "5-Year Job Growth")
display_msa = create_ordinal_metrics(display_msa, "Household_Growth_Tier", "5-Year Household Growth")
display_msa = create_ordinal_metrics(display_msa, "Population_Growth_Tier", "5-Year Population Growth")
display_msa = create_ordinal_metrics(display_msa, "Income_Growth_Tier", "5-Year Income Growth")
display_msa = create_ordinal_metrics(display_msa, "Vacancy_Rate_Tier", "Vacancy_Rate", ascending=True)
display_msa = create_ordinal_metrics(display_msa, "Price_Tier", "housecanary_median_price", ascending=True)


### Adjust BiggerScore v1 and v2 based on the tiers
def adjust_tier(
        df,
        tier_name
):
    
    df['BiggerScore'] = np.where(
        df[f'{tier_name}'] == 1,
        df['BiggerScore'] * 0.8,
        np.where(
            df[f'{tier_name}'] == 2,
            df['BiggerScore'] * 0.9,
            df['BiggerScore']
        )
    )

    return df

# display_msa = adjust_tier(display_msa, "Population_Growth_Tier")
display_msa = adjust_tier(display_msa, "Household_Growth_Tier")
display_msa = adjust_tier(display_msa, "Job_Growth_Tier")
display_msa = adjust_tier(display_msa, "Income_Growth_Tier")
display_msa = adjust_tier(display_msa, "Vacancy_Rate_Tier")
display_msa = adjust_tier(display_msa, "Price_Tier")

# Due to "punishing" metrics, the range of the scores decreases.
# We will arbitraily increase the range for presentability.
# Users may be more likely to trust market scores between 30%-75% than 10%-55%.
display_msa['BiggerScore'] += 0.15


# Create final rankings
display_msa = display_msa.sort_values("BiggerScore", ascending=False).reset_index(drop=True)
display_msa['Final_Rank'] = display_msa.index + 1


# Reorganize columns
display_msa.drop(columns=['msa_name','msa_code_bls'], inplace=True)
display_msa = display_msa[['msa_name_original','Final_Rank','BiggerScore'] +
                          [col for col in display_msa if 
                                (col != 'msa_name_original') &
                                (col != 'BiggerScore') & 
                                (col != 'Final_Rank') 
                                ]]

display_msa = display_msa.sort_values("BiggerScore", ascending=False).reset_index(drop=True)

# Save to output
display_msa.to_csv("outputs/biggerscore_v2.csv", index=False)

display_msa.head(10)

,msa_name_original,Final_Rank,BiggerScore,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,Population_Size_Category,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,hc_median_rent_upper_bound,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply,Median_Prop_Tax_Rate,Normalized Total Employment,5-Year Job Growth x Normalized,RANK_5-Year Household Growth,RANK_5-Year Population Growth,RANK_5-Year Job Growth,RANK_5-Year Job Growth x Normalized,RANK_Income,RANK_5-Year Income Growth,RANK_Unemployment_Rate,RANK_Permits_as_Percent_of_Total_Units,RANK_1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_ACS_1_Year_Rent-Price_Ratio,RANK_housecanary_rentpriceratio,RANK_1-Year Price Forecast,RANK_ACS_1_Year_Median_Price,RANK_insurance,RANK_Median_Prop_Tax_Rate,RANK_Vacancy_Rate,Corporate_Tax_Ranking,Corporate_Tax_Ranking_10_Year_Change,landlord_friendly_or_not,Cashflow_Rank_Sum,Cashflow_Score,Supply_Rank_Sum,Supply_Score,Demand_Rank_Sum,Demand_Score,Landlord_Friendliness_Modifier,Total_Rank_Sum,Job_Growth_Tier,Household_Growth_Tier,Population_Growth_Tier,Income_Growth_Tier,Vacancy_Rate_Tier,Price_Tier
0,"Spartanburg, SC",1,0.8325,43900,0.0979,0.0284,357277.0000,0.0976,0.0203,0.0963,233400.0000,0.5792,813.0000,0.3780,0.3483,151566.0000,3,176400.0000,0.0607,54171.0000,0.2034,5.4000,2023.0000,2824.0000,2.0000,2.0000,41.0000,2867.0000,98.5002,0.0698,0.0698,1.4301,0.0189,253800.0000,1596.0000,2000.0000,0.0063,0.0429,0.0211,SC,Metropolitan Statistical Area,1020.0000,3219.0000,0.0095,0.0044,0.0151,0.0917,302,339,235,267,201,184,46,58,315,173,207,210,234,173,316,187,6,6,Landlord Friendly,3534,0.6570,746,0.5068,5309,0.6170,0.1000,9589,3,4,4,3,3,3
1,"Oklahoma City, OK",2,0.8173,36420,0.0966,0.0165,1428923.0000,0.0557,0.0114,0.0885,244000.0000,0.5165,927.0000,0.3262,0.3799,608642.0000,5,716200.0000,0.0739,58398.6000,0.2214,3.3000,2023.0000,5573.0000,556.0000,756.0000,407.0000,6736.0000,82.7346,8.2542,11.2233,6.0422,0.0111,244700.0000,1523.0000,1897.0000,0.0062,0.0393,0.0055,OK,Metropolitan Statistical Area,2045.0000,7012.0000,0.0054,0.0084,0.0680,0.5026,298,271,268,343,279,206,311,138,247,240,197,184,219,1,152,209,4,7,Landlord Friendly,1794,0.3335,770,0.5231,6811,0.7916,0.1000,9375,3,4,3,3,3,3
2,"Springfield, MO",3,0.8111,44180,0.0808,0.0144,477056.0000,0.0482,0.0104,0.0570,242200.0000,0.6136,815.0000,0.3674,0.3365,208534.0000,3,238400.0000,0.0831,47032.9600,0.2495,3.4000,2023.0000,1580.0000,106.0000,171.0000,426.0000,2177.0000,72.5769,4.8691,7.8548,19.5682,0.0104,253800.0000,1572.0000,1934.0000,0.0062,0.0456,-0.0791,MO,Metropolitan Statistical Area,1548.0000,4114.0000,0.0040,0.0064,0.0212,0.1760,271,252,290,296,90,244,308,149,212,146,194,227,221,89,217,331,3,1,Landlord Friendly,2554,0.4748,722,0.4905,6012,0.6987,0.1000,9288,4,3,3,3,4,3
3,"Fort Wayne, IN",4,0.8013,23060,0.0583,0.0130,447882.0000,0.0439,0.0088,0.0672,222700.0000,0.6411,787.0000,0.2673,0.3534,189593.0000,3,237900.0000,0.0434,57032.0400,0.2145,4.1000,2023.0000,1680.0000,44.0000,50.0000,706.0000,2436.0000,68.9655,1.8062,2.0525,28.9819,0.0128,247900.0000,1604.0000,2052.0000,0.0065,0.0551,0.0338,IN,Metropolitan Statistical Area,1305.0000,3620.0000,0.0001,0.0059,0.0211,0.0917,221,245,206,265,251,194,179,110,116,189,226,284,250,134,244,295,12,16,Landlord Friendly,3068,0.5704,452,0.3071,5631,0.6545,0.1000,9151,3,3,3,3,4,3
4,"Huntsville, AL",5,0.7776,26620,0.1346,0.0198,4939

In [16]:
display_msa.sort_values("BiggerScore", ascending=False).head(10)['msa_name_original']

0                       Spartanburg, SC
1                     Oklahoma City, OK
2                       Springfield, MO
3                        Fort Wayne, IN
4                        Huntsville, AL
5             Salt Lake City-Murray, UT
6                          Columbia, MO
7                        Boise City, ID
8                          Richmond, VA
9    Fayetteville-Springdale-Rogers, AR
Name: msa_name_original, dtype: object

In [19]:
display_msa[['msa_name_original','BiggerScore','Final_Rank',
             'Cashflow_Score','Supply_Score','Demand_Score','housecanary_rentpriceratio'
]].sort_values("BiggerScore", ascending=False).head(30)

,msa_name_original,BiggerScore,Final_Rank,Cashflow_Score,Supply_Score,Demand_Score,housecanary_rentpriceratio
0,"Spartanburg, SC",0.8325,1,0.6570,0.5068,0.6170,0.0063
1,"Oklahoma City, OK",0.8173,2,0.3335,0.5231,0.7916,0.0062
2,"Springfield, MO",0.8111,3,0.4748,0.4905,0.6987,0.0062
3,"Fort Wayne, IN",0.8013,4,0.5704,0.3071,0.6545,0.0065
4,"Huntsville, AL",0.7776,5,0.5427,0.0639,0.7886,0.0052
5,"Salt Lake City-Murray, UT",0.7776,6,0.5771,0.2405,0.8791,0.0039
6,"Columbia, MO",0.7773,7,0.4705,0.6916,0.6119,0.0074
7,"Boise City, ID",0.7635,8,0.5436,0.1875,0.8803,0.0042
8,"Richmond, VA",0.7615,9,0.5131,0.3111,0.8465,0.0059
9,"Fayetteville-Springdale-Rogers, AR",0.7585,10,0.4032,0.1250,0.8307,0.0046


In [11]:
display_msa[['msa_name_original','BiggerScore','Final_Rank','BiggerScore_v2',
             'Final_Rank_v2','Cashflow_Score','Supply_Score','Demand_Score','housecanary_rentpriceratio'
]].sort_values("BiggerScore", ascending=False).head(30)

,msa_name_original,BiggerScore,Final_Rank,BiggerScore_v2,Final_Rank_v2,Cashflow_Score,Supply_Score,Demand_Score,housecanary_rentpriceratio
0,"Spartanburg, SC",0.8325,1,0.8541,1,0.6523,0.8560,0.6316,0.0063
1,"Oklahoma City, OK",0.8173,2,0.6858,7,0.3972,0.6712,0.7766,0.0062
2,"Springfield, MO",0.8111,3,0.6975,4,0.5036,0.5761,0.6971,0.0062
3,"Fort Wayne, IN",0.8013,4,0.6629,12,0.5993,0.3152,0.6535,0.0065
4,"Huntsville, AL",0.7776,5,0.5017,59,0.4872,0.0571,0.7910,0.0052
5,"Salt Lake City-Murray, UT",0.7776,6,0.5267,45,0.4577,0.3505,0.8867,0.0039
6,"Columbia, MO",0.7773,7,0.7776,2,0.5484,0.8397,0.6057,0.0074
7,"Boise City, ID",0.7635,8,0.5144,53,0.4388,0.3342,0.8902,0.0042
8,"Richmond, VA",0.7615,9,0.5719,32,0.5007,0.4457,0.8381,0.0059
9,"Fayetteville-Springdale-Rogers, AR",0.7585,10,0.4747,79,0.3502,0.1984,0.8258,0.0046


In [12]:
display_msa[display_msa['msa_name_original'].str.contains("Indiana")]

,msa_name_original,Final_Rank,BiggerScore,Final_Rank_v2,BiggerScore_v2,msa_code,5-Year Household Growth,1-Year Household Growth,Population,5-Year Population Growth,1-Year Population Growth,Vacancy_Rate,ACS_1_Year_Median_Price,5-Year Price Growth,ACS_1_Year_Median_Rent,5-Year Rent Growth,ACS_1_Year_Rent-Price_Ratio,Total_Units,Population_Size_Category,Jobs,5-Year Job Growth,Income,5-Year Income Growth,Unemployment_Rate,permit_data_for_year,sfh_permits,duplex_unit_permits,small_multifamily_unit_permits,commercial_multifamily_unit_permits,total_unit_permits,percent_sfh,percent_duplex,percent_small_multi,percent_large_multi,Permits_as_Percent_of_Total_Units,housecanary_median_price,housecanary_median_rent,hc_median_rent_upper_bound,housecanary_rentpriceratio,1-Year Price Forecast,1-Year Rent Forecast,state,metro_or_micro_area,median_prop_taxes_by_msa,insurance,1-Year_HH_Growth_Minus_Percent_New_Supply,Median_Prop_Tax_Rate,Normalized Total Employment,5-Year Job Growth x Normalized,RANK_5-Year Household Growth,RANK_5-Year Population Growth,RANK_5-Year Job Growth,RANK_5-Year Job Growth x Normalized,RANK_Income,RANK_5-Year Income Growth,RANK_Unemployment_Rate,RANK_Permits_as_Percent_of_Total_Units,RANK_1-Year_HH_Growth_Minus_Percent_New_Supply,RANK_ACS_1_Year_Rent-Price_Ratio,RANK_housecanary_rentpriceratio,RANK_1-Year Price Forecast,RANK_ACS_1_Year_Median_Price,RANK_insurance,RANK_Median_Prop_Tax_Rate,RANK_Vacancy_Rate,Corporate_Tax_Ranking,Corporate_Tax_Ranking_10_Year_Change,landlord_friendly_or_not,Cashflow_Rank_Sum,Cashflow_Score,Supply_Rank_Sum,Supply_Score,Demand_Rank_Sum,Demand_Score,Landlord_Friendliness_Modifier,Total_Rank_Sum,Total_Rank_Sum_v2,Job_Growth_Tier,Household_Growth_Tier,Population_Growth_Tier,Income_Growth_Tier,Vacancy_Rate_Tier,Price_Tier
29,"Indianapolis-Carmel-Greenwood, IN",30,0.6495,49,0.5218,26900,0.0832,0.0158,2088343.0000,0.0618,0.0096,0.0820,278200.0000,0.6442,1007.0000,0.3701,0.3620,888218.0000,5,1205500.0000,0.1008,55980.0800,0.0712,4.0000,2023.0000,7252.0000,202.0000,342.0000,4960.0000,12554.0000,57.7664,1.6090,2.7242,39.5093,0.0141,270000.0000,1759.0000,2210.0000,0.0065,0.0355,0.0138,IN,Metropolitan Statistical Area,2887.0000,3620.0000,0.0017,0.0104,0.1159,1.1688,276,286,312,353,239,55,203,103,148,207,231,147,172,137,122,232,12,16,Landlord Friendly,1383,0.4922,444,0.4022,5956,0.7458,0.1000,10061,4.2646,4,3,4,1,3,3
